In [13]:
from gpt import GPT, GPTConfig
device = 'cuda'
import torch
import torch.nn.functional as F
import tiktoken


In [9]:
trained_model = GPT.from_pretrained('gpt2')
untrined_model = model =  GPT(GPTConfig(vocab_size=50257))
enc = tiktoken.get_encoding('gpt2')

max_length = 25
number_return_sequences = 2

def inferenc(model):
    model.eval()

    prompt = 'translate to german: i am gpt'
    tokens = enc.encode(prompt)
    tokens = torch.tensor(tokens, dtype=torch.long)
    tokens = tokens.unsqueeze(0).repeat(number_return_sequences, 1)
    x = tokens.to(device)
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    while x.size(1) < max_length:
        with torch.no_grad():
            logits = model(x)[0] # B, T, vocab_size
            logits = logits[:, -1, :] # (B, vocab-size)
            # got the probabilities
            probs = F.softmax(logits, dim=-1 )
            # get prbablities
            topk_probs, topk_indicies =  torch.topk(probs, 50, dim=-1) # (5, 50)
            # Select top-k proababilits 
            ix = torch.multinomial(topk_probs, 1) #(B, 1)
            # gather the corresponding indecies
            xcol = torch.gather(topk_indicies, -1, ix) # (B, 1)
            x = torch.cat((x, xcol), dim=1)

        # print inducidual raws of output 
    for i in range(number_return_sequences):
        tokens = x [ i , :max_length].tolist()
        decoded = enc.decode(tokens)
        print(">", decoded)
    
#inferenc(trained_model)


loading weights from pretrained gpt: gpt2


In [10]:
def multiply_specific_layer(model, tensor, layer_name):
    # Check if the layer exists in the model
    if layer_name not in dict(model.named_parameters()):
        raise ValueError(f"Layer {layer_name} not found in the model")

    # Get the layer parameter (e.g., weight or bias)
    layer_param = dict(model.named_parameters())[layer_name]

    # Ensure the shapes of the layer and the tensor are compatible
    if layer_param.shape != tensor.shape:
        raise ValueError(f"Shape mismatch: Layer shape {layer_param.shape}, Tensor shape {tensor.shape}")

    # Perform the element-wise multiplication with the tensor
    with torch.no_grad():  # Ensure no gradients are tracked
        layer_param.mul_(tensor)  # In-place multiplication of the layer's weight

    return model

In [15]:
model = trained_model   # Assuming this is your model
tensor = torch.zeros_like(model.transformer.h[1].ln_2.weight)  # Example tensor with matching shape

layer_name = "transformer.h.1.ln_2.weight"  # The layer we want to modify
updated_model = multiply_specific_layer(model, tensor, layer_name)
inferenc(model.to(device))

> translate to german: i am gptv: Gpt: http://www.kern.de gpt
> translate to german: i am gpt for an in german: i hate gt /.

Anonymous


In [36]:
import torch.nn as nn
class GPTprover(nn.Module):
    def __init__(self, model, dict_key) -> None:
        super().__init__()
        self.model = model
        self.layer_name = dict_key
        self.l1 = nn.Parameter(torch.ones_like(model.state_dict()[dict_key].shape))
        
    def forward(self, x, y=None): 
        new_model = multiply_specific_layer(self.model, self.l1, self.layer_name)
        new_model.eval()
        logit, loss = new_model(x, y)
        loss = 0.5 * loss + 0.5 * torch.sum(torch.abs(self.l1))
        return logit, loss

        

In [39]:

import time
from gpt import DataLoaderLight
train_loader = DataLoaderLight(B=8, T=1024)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-10, betas=(0.9, 0.95), eps=1e-8)
#optimizer = model.configure_optimizer(weight_decay=0.1, learning_rate = 6e-4, device=device)
model = GPTprover(trained_model, "transformer.h.1.ln_2.weight")
model.to(device)
for step in range(500):
    t0 = time.time()
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad()
    with torch.autocast(device_type=device, dtype=torch.float16):
        logits, loss = model(x, y)
        #import code; code.interact(local=locals())
    loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    lr = 3e-4
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    optimizer.step()
    torch.cuda.synchronize()
    t1 = time.time()
    dt = (t1 -t0 ) * 1000 # time difference in milisecond
    tokens_per_sec = (train_loader.B * train_loader.T)/ (t1-t0)
    print(f"Step {step} | loss {loss.item()} | lr {lr: .4e}  | norm : {norm: .4f} |  dt: {dt: .2f}ms | tok/sec: {tokens_per_sec}")



LOaded 338025 tokens
1 epoch = 41 batches
Step 0 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  232.12ms | tok/sec: 35292.19341008793
Step 1 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  197.57ms | tok/sec: 41463.516857051145
Step 2 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  197.80ms | tok/sec: 41415.38813722146
Step 3 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  198.64ms | tok/sec: 41240.80850844207
Step 4 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  199.29ms | tok/sec: 41106.65603659908
Step 5 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  198.38ms | tok/sec: 41295.18151269392
Step 6 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  198.01ms | tok/sec: 41371.45536640988
Step 7 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  198.08ms | tok/sec: 41356.46677314794
Step 8 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  196.07ms | tok/sec: 41780.54875447782
Step 9 | loss nan | lr  3.0000e-04  | norm :  nan |  dt:  198.52ms | tok/sec: 41265.821128837

inferenc

In [ ]:
inferenc()

In [8]:
import torch

def multiply_trainable_weights(model1, model2):
    # Ensure that model2's parameters do not require gradients
    for param in model2.parameters():
        param.requires_grad = False

    # Iterate over both models' parameters
    for param1, param2 in zip(model1.parameters(), model2.parameters()):
        # Only multiply if param1 is trainable (i.e., requires_grad is True)
        if param1.requires_grad:
            # Ensure the shapes are the same
            if param1.shape == param2.shape:
                # Perform element-wise multiplication
                with torch.no_grad():  # Disable gradient computation for in-place operations
                    param1.mul_(param2)  # In-place multiplication of param1 by param2
            else:
                raise ValueError(f"Shape mismatch: {param1.shape} and {param2.shape}")

multiply_trainable_weights(untrined_model, trained_model)


In [12]:
untrined_model.to(device)
inferenc(untrined_model)
trained_model.to(device)
inferenc(trained_model)

> translate to german: i am gpt Picturecn migrants civilian errone concussion pled pledfuelVILLE doubted>>>>>>>> knowing MAY BlackBerry
> translate to german: i am gptitiveness eternity weboting planetbbFUN Log Dollarsale Kazakhpert Ragnarok Ripple681
> translate to german: i am gpt alexian, and this post is by kyle for a short interview
> translate to german: i am gptj (says) a little long ago (mockingly) because


In [44]:
def multiply_learnable_state_dicts(model, state_dict1, state_dict2):
    """
    Multiplies learnable parameters of two state dictionaries of a model item by item.

    Args:
        model (torch.nn.Module): The PyTorch model.
        state_dict1 (dict): First state dictionary.
        state_dict2 (dict): Second state dictionary.
        
    Returns:
        dict: A new state dictionary with learnable parameters multiplied.
    """
    # Initialize an empty dictionary to hold the results
    multiplied_state_dict = {}

    # Iterate over learnable parameters in the model
    for name, param in model.named_parameters():
        if name in state_dict1 and name in state_dict2:
            # Multiply only the learnable parameters (requires_grad=True)
            multiplied_state_dict[name] = state_dict1[name] * state_dict2[name]
        else:
            raise KeyError(f"Key {name} not found in both state dictionaries.")
    
    return multiplied_state_dict

In [42]:
import torch
import torch.nn as nn

# Function to set 50% of the weights to zero and 50% to random values
def modify_weights(model):
    with torch.no_grad():  # Ensure gradients aren't calculated
        for param in model.parameters():
            if param.requires_grad:  # Ensure we're working with trainable parameters
                # Get the current weight tensor
                weight_tensor = param.data
                
                # Flatten the weight tensor for easier manipulation
                flattened_weights = weight_tensor.view(-1)
                
                # Create a random mask that selects 50% of the elements to be zero
                mask = torch.rand(flattened_weights.shape) < 1.0
                
                # Set 50% of weights to zero
                flattened_weights[mask] = 1
                
                # Set the remaining weights to random values from a normal distribution
                flattened_weights[~mask] = torch.randn(flattened_weights[~mask].shape)
                
                # Reshape back to original shape
                param.data = flattened_weights.view(weight_tensor.shape)

# Function to compare the weights before and after modification
def check_weights_different(model, original_weights):
    all_different = True
    for param, original_param in zip(model.parameters(), original_weights):
        if torch.equal(param.data, original_param):
            all_different = False
            break
    return all_different

# Function to restore the original weights to the model
def load_original_weights(model, original_weights):
    with torch.no_grad():  # Ensure no gradients are calculated
        for param, original_param in zip(model.parameters(), original_weights):
            param.data.copy_(original_param)

# Example of how to use the functions
if __name__ == "__main__":
    # Define a simple model (you can replace this with your own model)
    model = GPT.from_pretrained('gpt2-medium')
    # Save a copy of the original weights for comparison and reloading later
    original_weights = [param.clone() for param in model.parameters()]

    # Modify the weights
    modify_weights(model)
    multiplied_state_dict = multiply_learnable_state_dicts(model, original_weights, )
    
    inferenc(model=model)
    
    # Check if the weights are different
    if check_weights_different(model, original_weights):
        print("Weights were successfully modified.")
    else:
        print("Weights were not modified.")
    
    # Restore the original weights
    load_original_weights(model, original_weights)
    model.to(device)
    inferenc(model)
    # Check if the weights are back to the original values
    if not check_weights_different(model, original_weights):
        print("Weights were successfully restored to the original.")
    else:
        print("Weights were not restored correctly.")

loading weights from pretrained gpt: gpt2-medium
> translate to german: i am gptK3*5"+0,?7EI;1=
> translate to german: i am gpt97B*#5OHM#A5/LB
Weights were successfully modified.
> translate to german: i am gpt3r2c2s.<|endoftext|>About

As we said last
> translate to german: i am gpt gp for german/fr on the net or any language on the
Weights were successfully restored to the original.


In [60]:
def flatten_weights_and_keys(state_dict):
    """
    Flattens the learnable weights from the state_dict into a 1D tensor and returns the flattened tensor
    and a list of parameter keys.

    Args:
        state_dict (dict): The state_dict of the model containing weights and other parameters.

    Returns:
        flattened_tensor (torch.Tensor): 1D tensor of all learnable parameters.
        keys (list of str): List of keys corresponding to each parameter.
    """
    flattened_weights = []
    param_keys = []

    for key, value in state_dict.items():
        flattened_weights.append(value.view(-1))  # Flatten each tensor
        param_keys.append(key)

    # Concatenate all the flattened parameters into one large tensor
    flattened_tensor = torch.cat(flattened_weights)

    return flattened_tensor, param_keys
model_hf = GPT.from_pretrained('gpt2-medium')
sd_hf = model_hf.state_dict()
flattened_weight = flatten_weights_and_keys(sd_hf)

loading weights from pretrained gpt: gpt2-medium


In [51]:
print(flattened_weight[0].shape)

torch.Size([431452160])


In [61]:
def load_flattened_weights(flattened_tensor, model, param_keys):
    """
    Reorganizes the flattened tensor into the original structure and loads the weights into the model.

    Args:
        flattened_tensor (torch.Tensor): 1D tensor containing the flattened parameters.
        model (torch.nn.Module): The model into which the weights will be loaded.
        param_keys (list of str): List of parameter keys to help map the flattened tensor to the model.
    """
    state_dict = model.state_dict()  # Get the current state dict of the model
    current_index = 0  # Keep track of the position in the flattened tensor

    for key in param_keys:
        param = state_dict[key]
        param_size = param.numel()  # Get the number of elements in this parameter

        # Slice the corresponding part from the flattened tensor
        param_flat = flattened_tensor[current_index:current_index + param_size]

        # Reshape it back to the original size and load it into the model's state dict
        state_dict[key].copy_(param_flat.view_as(param))

        # Update the current index
        current_index += param_size

    # Load the updated state dict back into the model
    model.load_state_dict(state_dict)

model = load_flattened_weights(flattened_weight[0], model, flattened_weight[1] )

inferenc(model)

AttributeError: '_IncompatibleKeys' object has no attribute 'state_dict'